In [1]:
# imports
import requests
import pandas as pd
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
# Load bike station data from exported .csv (See citybikes.ipynb)
station_data = pd.read_csv('../data/citybike_stations.csv')

# Format the latitude & longitude of each station for use in our API calls
station_coordinates = []
for index, row in station_data.iterrows():
    # Gather the lat and long for each station
    lat = row['latitude']
    lon = row['longitude']
    # Format the values into a single string
    formatted_coords = f"{lat},{lon}"
    # Add it to our coordinate list
    station_coordinates.append(formatted_coords)

# Headers & parameters needed for FourSquare
headers = {"Accept": "application/json"}
headers['Authorization'] = os.environ["FOURSQUARE_API_KEY"]
radius_from_location = 1000 # Distance of POI from bike station location (in meters)

# Send an API request for every pair of station coordinates to compile POI data
poi_data = []
for station_coord in station_coordinates:
    # Create the API call using the bike station location and provided search radius
    request_url = "https://api.foursquare.com/v3/places/search?ll=" + station_coord + "&radius=" + str(radius_from_location)
    response = requests.get(request_url, headers=headers)
    poi_data.append(response.json())

Put your parsed results into a DataFrame

In [9]:
# NOTE:seperate code block so I don't have to keep calling API
# There may be a better way to do the below, but I had a lot of issues getting this to work so I am leaving as-is :D

# Load the compiled data into Pandas dataframe 
poi_data_df = pd.DataFrame(poi_data)

# Use the sum function to flatten the results column (ie stop it from being a list of lists)
poi_data_flattened = sum(poi_data_df['results'], [])

# Load the organized data back into a dataframe
poi_data_df = pd.DataFrame(poi_data_flattened)

# Display results
poi_data_df

,fsq_id,categories,chains,distance,geocodes,link,location,name,related_places,timezone
0,54c7cade498e300defb0985f,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],31,"{'main': {'latitude': 55.858485, 'longitude': ...",/v3/places/54c7cade498e300defb0985f,"{'address': '6 Wilson St', 'admin_region': 'Sc...",Wilson Street Pantry,{},Europe/London
1,4bc35be22a89ef3b08e7f488,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...",[],34,"{'main': {'latitude': 55.858078, 'longitude': ...",/v3/places/4bc35be22a89ef3b08e7f488,"{'address': '44 Candleriggs', 'admin_region': ...",The Dhabba,{},Europe/London
2,4b9cecbcf964a520bd8236e3,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],43,"{'main': {'latitude': 55.858429, 'longitude': ...",/v3/places/4b9cecbcf964a520bd8236e3,"{'address': '71-73 Albion St', 'admin_region':...",Merchant Square,{'children': [{'fsq_id': '595f841bf79faa623f93...,Europe/London
3,4ba0e8c3f964a520508637e3,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...",[],98,"{'main': {'latitude': 55.85886, 'longitude': -...",/v3/places/4ba0e8c3f964a520508637e3,"{'admin_region': 'Scotland', 'country': 'GB', ...",City Halls,{},Europe/London
4,554cdd3c498ee17d5a870117,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...",[],144,"{'main': {'latitude': 55.859412, 'longitude': ...",/v3/places/554cdd3c498ee17d5a870117,"{'address': '127 Candleriggs', 'admin_region':...",Spitfire Espresso,{},Europe/London
...,...,...,...,...,...,...,...,...,...,...
1051,06b2e83c632e418806153b09,"[{'id': 17010, 'name': 'Used Car Dealership', ...",[],421,"{'main': {'latitude': 55.845601, 'longitude': ...",/v3/places/06b2e83c632e418806153b09,"{'address': '136 Boden St', 'admin_region': 'S...",London Cab Sales,{},Europe/London
1052,c6e7c00cef054a0a92e80063,"[{'id': 16037, 'name': 'Playground', 'icon': {...",[],543,"{'main': {'latitude': 55.842511, 'longitude': ...",/v3/places/c6e7c00cef054a0a92e80063,"{'address': '421 Baltic St', 'admin_region': '...",Baltic Street Adventure Playground,{},Europe/London
1053,4c1ae6aeb9f876b044847946,"[{'id': 12072, 'name': 'Police Station', 'icon...",[],557,"{'main': {'latitude': 55.848546, 'longitude': ...",/v3/places/4c1ae6aeb9f876b044847946,"{'address': '851 London Rd', 'admin_region': '...",London Road Police Station,{},Europe/London
1054,5bc4936d898bdc002bfa5d46,"[{'id': 15003, 'name': 'Alternative Medicine C...",[],666,"{'drop_off': {'latitude': 55.840302, 'longitud...",/v3/places/5bc4936d898bdc002bfa5d46,"{'address': '45 Auckland Wynd', 'admin_region'...",Certified Professional Never Binge Again Maste...,{},Europe/London


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [17]:
#selected_station_data = all_station_data[['free_bikes','latitude','longitude']].copy()
selected_poi_data_df = poi_data_df[['name','categories','location']].copy()

# Display results
selected_poi_data_df

,name,categories,location
0,Wilson Street Pantry,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...","{'address': '6 Wilson St', 'admin_region': 'Sc..."
1,The Dhabba,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...","{'address': '44 Candleriggs', 'admin_region': ..."
2,Merchant Square,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...","{'address': '71-73 Albion St', 'admin_region':..."
3,City Halls,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...","{'admin_region': 'Scotland', 'country': 'GB', ..."
4,Spitfire Espresso,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...","{'address': '127 Candleriggs', 'admin_region':..."
...,...,...,...
1051,London Cab Sales,"[{'id': 17010, 'name': 'Used Car Dealership', ...","{'address': '136 Boden St', 'admin_region': 'S..."
1052,Baltic Street Adventure Playground,"[{'id': 16037, 'name': 'Playground', 'icon': {...","{'address': '421 Baltic St', 'admin_region': '..."
1053,London Road Police Station,"[{'id': 12072, 'name': 'Police Station', 'icon...","{'address': '851 London Rd', 'admin_region': '..."
1054,Certified Professional Never Binge Again Maste...,"[{'id': 15003, 'name': 'Alternative Medicine C...","{'address': '45 Auckland Wynd', 'admin_region'..."


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating